In [ ]:
from langchain.document_loaders.pdf_loader import PDFLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
import gradio as gr
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. 读取PDF文件并将其内容转换为文本。
pdf_loader = PDFLoader()
document = pdf_loader.load('your_pdf_file.pdf')
text = document.get_text()

# 2. 将文本切分为小块，确保每块都不超过token的最大长度限制。
# 这里我假设你有一个名为tokenize的函数可以进行这个操作
tokens = tokenize(text, max_length=512)

# 3. 使用embedding模型将文本块转换为向量，并将这些向量存储到向量数据库中。
embedder = AzureOpenAIEmbeddings()
vectors = [embedder.embed(token) for token in tokens]

df = pd.DataFrame({'tokens': tokens, 'embedding_vec': vectors})

# 4. 使用Gradio开发用户界面，允许用户输入查询并执行语义搜索。

def search_reviews(df, product_description, n=3, pprint=True):
    product_embedding = embedder.embed(product_description)
    df["similarity"] = df.embedding_vec.apply(lambda x: cosine_similarity([x], [product_embedding])[0][0])

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .tokens.str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

iface = gr.Interface(fn=search_reviews, inputs=["dataframe", "text"], outputs="text")
iface.launch()